# Low-level interface for developers

In [1]:
import pandas
pandas.options.display.max_rows = 8
from cameo import load_model
model = load_model('iJO1366')

## cameo vs. cobrapy

### Importing a model

cobrapy (load a model in SBML format):

In [2]:
from cobra.io import read_sbml_model
cobrapy_model = read_sbml_model('../tests/data/EcoliCore.xml')

cameo (load models from different formats):

In [3]:
from cameo import load_model
# read SBML model
model = load_model('../tests/data/EcoliCore.xml')
# ... or read a pickled model
model = load_model('../tests/data/iJO1366.pickle')
# ... or just import a model by ID from http://darwin.di.uminho.pt/models
iAF1260 = load_model('iAF1260')

### Solving models

When optimizing models with cobrapy, the status of a returned solution needs to be checked to figure out if the solver found and optimal solution.

In [26]:
cobrapy_solution = cobrapy_model.optimize()
if cobrapy_solution.status == 'optimal':
    pass  # do something

cameo will raise an exception if non-optimal solution was found. Therefore, `try-except-else-finally` statements should be used in code that utilizes `cameo.solver_based_model.SolverBasedModel.solve`.

In [17]:
model.solve()  # no exception
cp_model = model.copy()
# demand a very high biomass production rendering the model infeasible
cp_model.reactions.Ec_biomass_iJO1366_core_53p95M.lower_bound = 1000

from cameo.exceptions import SolveError
try:
    solution = cp_model.solve()
except SolveError as e:
    print "A non-optimal solution was returned by the solver"
    print e
else:
    pass  # do something about it ...

A non-optimal solution was returned by the solver
Solving model iJO1366 did not return an optimal solution. The returned solution status is "infeasible"


It is important to note that cameo models still provide `optimize` to maintain
compatibility with cobrapy but we discourage its use and hide it from the user.

In [22]:
print model.optimize
'optimize' in dir(model)

<bound method SolverBasedModel.optimize of <SolverBasedModel iJO1366 at 0x10f2dd6d0>>


False

### Solutions

`cameo.solve` normally returns a lazy version of a cobrapy solution that is much more effiecient in retrieving solution status and objective value.

In [34]:
%time cobrapy_model.optimize()
%time model.solve()

CPU times: user 1.41 ms, sys: 28 µs, total: 1.44 ms
Wall time: 1.45 ms
CPU times: user 1.57 ms, sys: 77 µs, total: 1.64 ms
Wall time: 1.72 ms


In [29]:
print type(cobrapy_solution)
print type(solution)

<class 'cobra.core.Solution.Solution'>
<class 'cameo.solver_based_model.LazySolution'>


## Formulating and solving optimization problems

http://optlang.readthedocs.org/en/latest/

## IO

### Importing models

cameo has a single function (`load_model`) that can import models stored in different formats.

#### SBML

#### Models available on the web

cameo provides an interface to models hosted online by the [University of Minho](http://darwin.di.uminho.pt/models).

In [19]:
from cameo.webmodels import index_models
index_models()

,id,name,doi,author,year,formats,organism,taxonomy
0,1,iJR904,10.1186/gb-2003-4-9-r54,Reed,2003,[sbml],Escherichia coli str. K12 substr. MG1655,Bacteria; Proteobacteria; Gammaproteobacteria;...
1,2,iAF1260,10.1038/msb4100155,Feist,2007,[sbml],Escherichia coli str. K12 substr. MG1655,Bacteria; Proteobacteria; Gammaproteobacteria;...
2,3,iMM904,10.1186/1752-0509-3-37,Mo,2007,[sbml],Saccharomyces cerevisiae,Eukaryota; Opisthokonta; Fungi; Dikarya; Ascom...
3,4,iJP815,10.1371/journal.pcbi.1000210,Puchalka,2008,[sbml],Pseudomonas putida str. KT2440,Bacteria; Proteobacteria; Gammaproteobacteria;...
...,...,...,...,...,...,...,...,...
132,133,iTT548,10.1186/1475-2859-13-61,Na-Rae Lee,2014,"[sbml, excel]",Thermus thermophilus,Bacteria; Deinococcus-Thermus; Deinococci; The...
133,134,EctoGEM-1.0,10.1111/tpj.12627,Prigent,2014,[sbml],Ectocarpus siliculosus,Eukaryota; Stramenopiles; PX clade; Phaeophyce...
134,135,iMF721,10.1111/1462-2920.12513,Fondi,2014,[sbml],Pseudoalteromonas haloplanktis TAC125,Bacteria; Proteobacteria; Gammaproteobacteria;...
135,136,Arabidopsis core model,10.1104/pp.114.235358,Arnold,2014,[sbml],Arabidopsis thaliana,Eukaryota; Viridiplantae; Streptophyta; Strept...


If `load_model` is provided with a valid model name, it will automatically download the model in SBML format and import it.

In [21]:
load_model('iMM904')

<SolverBasedModel iMM904 at 0x11962bd90>

## Strain-design

All strain-design methods have been coded as python classes whose instances are either callable or provide `run` methods.

### Predicting heterologous pathways

In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [3]:
from cameo.strain_design.pathway_prediction import PathwayPredictor

In [4]:
pathway_predictor = PathwayPredictor(model)

DEBUG:cameo.strain_design.pathway_prediction:Loading default universal model.
DEBUG:cameo.strain_design.pathway_prediction:Adding reactions from universal model to host model.
DEBUG:cameo.strain_design.pathway_prediction:Adding adapter reactions to connect model with universal model.
DEBUG:cameo.strain_design.pathway_prediction:Adding switches.
DEBUG:cameo.strain_design.pathway_prediction:Setting minimization of switch variables as objective.


In [5]:
predictions = pathway_predictor.run(product='propane-1,3-diol', timeout=60)

DEBUG:cameo.strain_design.pathway_prediction:Predicting pathway No. 1
DEBUG:cameo.strain_design.pathway_prediction:Pathway predicted: 1 H(+) + 1 3-hydroxypropanal + 1 NADPH = 1 propane-1,3-diol + 1 NADP(+)	1 glycerol = 1 3-hydroxypropanal + 1 H2O
DEBUG:cameo.strain_design.pathway_prediction:Adding integer cut.
DEBUG:cameo.strain_design.pathway_prediction:Predicting pathway No. 2
DEBUG:cameo.strain_design.pathway_prediction:Pathway predicted: 1 H(+) + 1 NADH(2-) + 1 3-hydroxypropanal = 1 propane-1,3-diol + 1 NAD(+)	1 glycerol = 1 3-hydroxypropanal + 1 H2O
DEBUG:cameo.strain_design.pathway_prediction:Adding integer cut.
DEBUG:cameo.strain_design.pathway_prediction:Predicting pathway No. 3
DEBUG:cameo.core.solver_based_model:No optimal solution found. Turning on presolve.
DEBUG:cameo.core.solver_based_model:hey, infeasible, infeasible
DEBUG:cameo.strain_design.pathway_prediction:No pathway could be predicted. Terminating pathway predictions.


In [14]:
pathway[0].annotation

{'Description': '1 H(+) + 1 3-hydroxypropanal + 1 NADPH = 1 propane-1,3-diol + 1 NADP(+)',
 'EC': '1.1.1'}

In [17]:
for pathway in predictions:
    print(80*"#")
    for reaction in pathway:
        try:
            print(reaction.annotation['Description'])
        except KeyError:
            pass

################################################################################
1 H(+) + 1 3-hydroxypropanal + 1 NADPH = 1 propane-1,3-diol + 1 NADP(+)
1 glycerol = 1 3-hydroxypropanal + 1 H2O
################################################################################
1 H(+) + 1 NADH(2-) + 1 3-hydroxypropanal = 1 propane-1,3-diol + 1 NAD(+)
1 glycerol = 1 3-hydroxypropanal + 1 H2O


### Deterministic methods

### Heuristic methods

## Best practices

### Avoid copies at all costs

Copying models is expensive!

In [16]:
%time model_copy = model.copy()

CPU times: user 3.17 s, sys: 84.7 ms, total: 3.26 s
Wall time: 3.25 s


To avoid excessive copying of models, we suggest the following utility included in cameo. For example, temporary reaction deletion, solve the model and magically returns to it original state.

In [2]:
from functools import partial
from cameo.util import TimeMachine
with TimeMachine() as tm:
    tm

TimeMachine performs `do` steps immediately and stores all corresponding `undo` actions in order. Running all `undo` steps can be achieved with `TimeMachine.reset()` (wich is run)
We recommend using `TimeMachine` in conjunction with `with`, as in case of an unforseen exception all `undo` steps will be run and manipulated models will return to their original states. This is more elegant than using explicit `try-except-else-finally` statements.

### IPython notebook

Click [here](http://nbviewer.ipython.org/github/biosustain/cameo/blob/devel/docs/cameo_low_level_interface.ipynb) to download this page as an IPython notebook.